1.  Сколько пользователей, которые совершили покупку только один раз? 

Другими словами, сколько customer_unique_id, у которых order_status = Delivered только один раз.
Для ответа нужно соединить таблицы olist_customers_dataset и olist_orders_dataset, взять выборку order_status = Delivered, 
сгруппировать по customer_unique_id и посчитать количество заказов (например, по той же колонке order_status). Далее отобрать
строки с одним заказом.

In [1]:
# Блок импорта библиотек
import pandas as pd
import requests
from urllib.parse import urlencode
import numpy as np
import plotly.express as px
from datetime import timedelta
import matplotlib.pyplot as plt
from datetime import datetime

Получим прямые ссылки на все датафреймы

olist_customers_datase.csv — таблица с уникальными идентификаторами пользователей

olist_orders_dataset.csv — таблица заказов

olist_order_items_dataset.csv — товарные позиции, входящие в заказы


In [2]:
# olist_customers_datase.csv
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key_1 = 'https://disk.yandex.ru/d/FUi5uSd6BfG_ig' 
# Получаем загрузочную ссылку
final_url_1 = base_url + urlencode(dict(public_key=public_key_1))
response = requests.get(final_url_1)
download_url_1 = response.json()['href']
olist_customers_dataset = pd.read_csv(download_url_1, sep=',')

# olist_orders_dataset.csv
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key_2 = 'https://disk.yandex.ru/d/t9Li4JOfgxuUrg' 
# Получаем загрузочную ссылку
final_url_2 = base_url + urlencode(dict(public_key=public_key_2))
response = requests.get(final_url_2)
download_url_2 = response.json()['href']
olist_orders_dataset = pd.read_csv(download_url_2, sep=',')

# olist_order_items_dataset.csv
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key_3 = 'https://disk.yandex.ru/d/Gbt-yAcQrOe3Pw' 
# Получаем загрузочную ссылку
final_url_3 = base_url + urlencode(dict(public_key=public_key_3))
response = requests.get(final_url_3)
download_url_3 = response.json()['href']
olist_order_items_dataset = pd.read_csv(download_url_3, sep=',')

In [3]:
customers_and_orders = olist_customers_dataset\
 .merge(olist_orders_dataset, on='customer_id')

In [4]:
customers_and_orders.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06 00:00:00
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13 00:00:00
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10 00:00:00
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15 00:00:00


In [5]:
customers_and_orders.shape

(99441, 12)

In [6]:
customers_and_orders_delivered = customers_and_orders.query('order_status == "delivered"') \
 .groupby('customer_unique_id', as_index=False) \
 .agg({'order_status':'count'}) \
 .rename(columns={'order_status': 'order_number'}) \
 .sort_values('order_number', ascending = False)

In [12]:
customers_and_orders_delivered.head(10)

,customer_unique_id,order_number
51431,8d50f5eadf50201ccdcedfb9e2ac8455,15
22779,3e43e6105506432c953e165fb2acf44c,9
36706,6469f99c1f9dfae7733b25662e7f1782,7
73921,ca77025e7201e3b30c44b472ff346268,7
10060,1b6c7548a2a1f9037c1fd3ddfed95f33,7
26257,47c1a3033b8b77b3ab6e109eb4d5fdf3,6
80538,dc813062e0fc23409cd255f7f53c7074,6
6964,12f5d6e1cbf93dafd9dcc19095df0b3d,6
87885,f0e310a6839dce9de1638e0fe5ab282a,6
36500,63cfc61cee11cbe306bff5857d00bfe4,6


In [13]:
customers_and_orders_delivered.query('order_number == 1').value_counts()

customer_unique_id                order_number
0000366f3b9a7992bf8c76cfdf3221e2  1               1
aa5cdee8983dd08a8aad86ded1459971  1               1
aa884350c48f728088dd1e84ef7e8309  1               1
aa8804bf7d630692c09b03528f1fdedd  1               1
aa87d81170176d34f2aa2965c4ed431c  1               1
                                                 ..
551a0e417b10f992a724b436f2a5588c  1               1
551a09ef128e5436c555ecc43cfca103  1               1
5519d48c02e36392f44e56019db3b34d  1               1
5518a112dff474788c0d52cd4688acff  1               1
ffffd2657e2aad2907e67c3e9daecbeb  1               1
Length: 90557, dtype: int64

Ответ: 90557 пользователей совершили покупку только один раз.